In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
import os, json, cv2, random
from numpy import load

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer

In [2]:
# Rami's case:
JSONpath = "/media/opendr/7dd9f6a3-cbe2-481b-8fbd-473c4c7f2b5d/home/amir/rami_test/test/test2/test_aug_from_json/augumentations/Level1/augumentedJSON.json"
classes = ["allen key","screwdriver","big bolt","pipe", "medium bolt", "small bolt", "bolt hole", "cover", "elbow", "elbow target", "cover target", "metallic box"]
imgPath = "/media/opendr/7dd9f6a3-cbe2-481b-8fbd-473c4c7f2b5d/home/amir/rami_test/test/test2/test_aug_from_json/augumentations/Level1/"
programName = 'engine_'
hasKeypoints = False
hasSegmentationAndBBox = True
onlyBBox = False

if hasKeypoints == True:
    maxNumOfKeypoints = 16  ## give only if your dataset has keypoints

with open(JSONpath) as f:
    imgs_anns = json.load(f)

### Without segmentations
def get_dicts(img_dir,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox):
    dataset_dicts = []
    for idx, v in enumerate(imgs_anns):
      record = {}
      filename = os.path.join(img_dir, v["image"])
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      #### code for key points start ###
      if "kp-1" in v:
        print("hello")
        kps = v["kp-1"]
        obkps=[]
        countOfKeypoints = len(kps)
        for _, kp in enumerate(kps):
          x_kp = kp["x"] * width/100
          y_kp = kp["y"] * height/100
          v_kp = 1 #look at keypoints, it is not visible but labeled
          obkps.append(x_kp)
          obkps.append(y_kp)
          obkps.append(v_kp)
        
        if countOfKeypoints < maxNumOfKeypoints:
          for kp in range(0,(maxNumOfKeypoints-countOfKeypoints)):
            obkps.append(0)
            obkps.append(0)
            obkps.append(0)
      #else:  ## Remove if your dataset does not have keypoints
      #  obkps=[]
      #  for kp in range(0,maxNumOfKeypoints):
      #    obkps.append(0)
      #    obkps.append(0)
      #    obkps.append(0)
        #### code for key points ends ###
      if "label" in v:
        annos = v["label"]
      objs = []
      noSegmentationPoints = 0
      for _, anno in enumerate(annos):
        if "label" in v:              
          getPoints = np.array(anno["points"])
          if len(getPoints) != 0:
            getPoints[:,0] = getPoints[:,0] * width/100
            getPoints[:,1] = getPoints[:,1] * height/100
            px = getPoints[:,0]
            py = getPoints[:,1]
            poly = (getPoints.flatten()).tolist()
          else:
            noSegmentationPoints = 1

        for ind in range(0,len(classes)):
          if anno["polygonlabels"][0] == classes[ind]:
            catId= ind        
        if ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 0):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [],
              "category_id": catId,
              "keypoints": obkps,
          }
        elif ("label" in v) and (noSegmentationPoints == 0):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": catId,
              "keypoints": [],
          }
        elif ("kp-1" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": [],
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": []
          }
        elif ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": BoxMode.XYXY_ABS,
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": catId
          }
        objs.append(obj)
      record["annotations"] = objs
      dataset_dicts.append(record)
    return dataset_dicts


In [3]:
#def get_dicts(Images,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox):
def get_dicts(img_dir,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox):
    dataset_dicts = []
    for idx, v in enumerate(imgs_anns):
      record = {}
      filename = os.path.join(img_dir, v["image"])
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      #### code for key points start ###
      if ("kp-1" in v) and (hasKeypoints == True):
        kps = v["kp-1"]
        obkps=[]
        countOfKeypoints = len(kps)
        for _, kp in enumerate(kps):
          x_kp = kp["x"] * width/100
          y_kp = kp["y"] * height/100
          v_kp = 1 #look at keypoints, it is not visible but labeled
          obkps.append(x_kp)
          obkps.append(y_kp)
          obkps.append(v_kp)
        
        if countOfKeypoints < maxNumOfKeypoints:
          for kp in range(0,(maxNumOfKeypoints-countOfKeypoints)):
            obkps.append(0)
            obkps.append(0)
            obkps.append(0)
      elif hasKeypoints == True:  ## Remove if your dataset does not have keypoints
        obkps=[]
        for kp in range(0,maxNumOfKeypoints):
          obkps.append(0)
          obkps.append(0)
          obkps.append(0)
        #### code for key points ends ###
      if "label" in v and (hasSegmentationAndBBox == True):
        annos = v["label"]
      objs = []
      noSegmentationPoints = 0
      for _, anno in enumerate(annos):
        if "label" in v:              
          getPoints = np.array(anno["points"])
          if len(getPoints) != 0:
            getPoints[:,0] = getPoints[:,0] * width/100
            getPoints[:,1] = getPoints[:,1] * height/100
            px = getPoints[:,0]
            py = getPoints[:,1]
            poly = (getPoints.flatten()).tolist()
          else:
            noSegmentationPoints = 1   ## If any image does not contains segmentations

        for ind in range(0,len(classes)):
          if anno["polygonlabels"][0] == classes[ind]:
            catId= ind        
        if ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 0):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": catId,
              "keypoints": obkps,
          }
        elif ("label" in v) and (noSegmentationPoints == 0) and (onlyBBox == False) and (hasSegmentationAndBBox == True):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": catId,
              "keypoints": [],
          }
        elif ("label" in v) and (noSegmentationPoints == 0) and onlyBBox == True:
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [],
              "category_id": catId,
              "keypoints": [],
          }
        elif ("kp-1" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": [],
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": []
          }
        elif ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": BoxMode.XYXY_ABS,
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": catId
          }
        objs.append(obj)
      record["annotations"] = objs
      dataset_dicts.append(record)
    return dataset_dicts


In [4]:
for d in ["train"]:
    DatasetCatalog.register(programName + d, lambda d=d: get_dicts(imgPath,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox))
    MetadataCatalog.get(programName + d).set(thing_classes=classes)
    MetadataCatalog.get(programName + d).set(keypoint_names = ["kp1","kp2","kp3","kp4","kp5","kp6","kp7","kp8","kp9","kp10","kp11","kp12","kp13","kp14","kp15","kp16"])
    MetadataCatalog.get(programName + d).set(keypoint_flip_map = [])
engine_metadata = MetadataCatalog.get(programName+"_train")

In [5]:
for d in random.sample(get_dicts(imgPath,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox), 2):
#for d in dataset_dicts:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=engine_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("output",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

cfg.DATASETS.TRAIN = (programName+"train",)
cfg.MODEL.MASK_ON = True
#cfg.MODEL.KEYPOINT_ON = True
cfg.DATASETS.TEST = ()  
cfg.DATALOADER.NUM_WORKERS = 2

cfg.SOLVER.IMS_PER_BATCH = 2
#cfg.MODEL.BACKBONE.FREEZE_AT = 2
cfg.SOLVER.BASE_LR = 0.0004
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.GAMMA = 0.008
cfg.SOLVER.MOMENTUM = 0.95
cfg.SOLVER.MAX_ITER = 10000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
#cfg.MODEL.SEM_SEG_HEAD.NORM = "GN"
#cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 4
#cfg.MODEL.ROI_KEYPOINT_HEAD.NORMALIZE_LOSS_BY_VISIBLE_KEYPOINTS = True
cfg.MODEL.ROI_KEYPOINT_HEAD.LOSS_WEIGHT = 1.0
#cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = maxNumOfKeypoints
#cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((maxNumOfKeypoints, 1), dtype=float).tolist()

#Rami's case:
cfg.OUTPUT_DIR = "/media/opendr/7dd9f6a3-cbe2-481b-8fbd-473c4c7f2b5d/home/amir/rami_test/test/test2/test_aug_from_json/outputAfterPreprocessingv3"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[04/04 10:12:40 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[04/04 10:17:25 d2.data.build]: Removed 0 images with no usable annotations. 19008 images left.
[04/04 10:17:26 d2.data.build]: Distribution of instances among all 12 categories:
|   category   | #instances   |   category   | #instances   |   category   | #instances   |
|:------------:|:-------------|:------------:|:-------------|:------------:|:-------------|
|  allen key   | 2640         | screwdriver  | 2376         |   big bolt   | 14784        |
|     pipe     | 3080         | medium bolt  | 4400         |  small bolt  | 8008         |
|  bolt hole   | 10824        |    cover     | 4224         |    elbow     | 2640         |
| elbow target | 2024         | cover target | 1848         | metallic box | 2640         |
|              |              |              |              |              |              |
|    total     | 59488        |              |              |              |              |
[04/04 10:17:26 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in traini

2024-04-04 10:17:27.108097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 10:17:27.364448: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/opendr/Gaurang/environment/lib/python3.8/site-packages/cv2/../../lib64:/home/opendr/cobot_ws/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2024-04-04 10:17:27.364503: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-04 10:17:28.404419: W tensorflow/compiler/xla/stream

[04/04 10:17:29 d2.engine.train_loop]: Starting training from iteration 0


/home/opendr/Gaurang/environment/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


...

[04/04 10:56:16 d2.utils.events]:  eta: 0:16:13  iter: 7059  total_loss: 0.1775  loss_cls: 0.01819  loss_box_reg: 0.07334  loss_mask: 0.1065  loss_rpn_cls: 0.0002175  loss_rpn_loc: 0.004977  time: 0.3288  data_time: 0.0036  lr: 0.0004  max_mem: 2771M
[04/04 10:56:23 d2.utils.events]:  eta: 0:16:06  iter: 7079  total_loss: 0.1594  loss_cls: 0.02044  loss_box_reg: 0.05377  loss_mask: 0.0936  loss_rpn_cls: 0.0001128  loss_rpn_loc: 0.006836  time: 0.3288  data_time: 0.0034  lr: 0.0004  max_mem: 2771M
[04/04 10:56:29 d2.utils.events]:  eta: 0:15:59  iter: 7099  total_loss: 0.1906  loss_cls: 0.02245  loss_box_reg: 0.05978  loss_mask: 0.1141  loss_rpn_cls: 0.000183  loss_rpn_loc: 0.008765  time: 0.3288  data_time: 0.0034  lr: 0.0004  max_mem: 2771M
[04/04 10:56:36 d2.utils.events]:  eta: 0:15:52  iter: 7119  total_loss: 0.2393  loss_cls: 0.03056  loss_box_reg: 0.08031  loss_mask: 0.1179  loss_rpn_cls: 0.0002836  loss_rpn_loc: 0.009249  time: 0.3288  data_time: 0.0034  lr: 0.0004  max_mem: 277

[04/04 10:59:48 d2.utils.events]:  eta: 0:12:40  iter: 7699  total_loss: 0.2153  loss_cls: 0.02241  loss_box_reg: 0.06631  loss_mask: 0.1171  loss_rpn_cls: 0.0004929  loss_rpn_loc: 0.01013  time: 0.3289  data_time: 0.0034  lr: 0.0004  max_mem: 2771M
[04/04 10:59:55 d2.utils.events]:  eta: 0:12:34  iter: 7719  total_loss: 0.1941  loss_cls: 0.01966  loss_box_reg: 0.05832  loss_mask: 0.1089  loss_rpn_cls: 0.0001869  loss_rpn_loc: 0.005287  time: 0.3289  data_time: 0.0034  lr: 0.0004  max_mem: 2771M
[04/04 11:00:01 d2.utils.events]:  eta: 0:12:27  iter: 7739  total_loss: 0.1369  loss_cls: 0.01804  loss_box_reg: 0.04174  loss_mask: 0.08803  loss_rpn_cls: 0.0001518  loss_rpn_loc: 0.007415  time: 0.3289  data_time: 0.0033  lr: 0.0004  max_mem: 2771M
[04/04 11:00:08 d2.utils.events]:  eta: 0:12:20  iter: 7759  total_loss: 0.1915  loss_cls: 0.02338  loss_box_reg: 0.07277  loss_mask: 0.1047  loss_rpn_cls: 0.000123  loss_rpn_loc: 0.006585  time: 0.3289  data_time: 0.0034  lr: 0.0004  max_mem: 277

[04/04 11:03:22 d2.utils.events]:  eta: 0:09:08  iter: 8339  total_loss: 0.196  loss_cls: 0.01623  loss_box_reg: 0.04737  loss_mask: 0.1112  loss_rpn_cls: 0.0002914  loss_rpn_loc: 0.005411  time: 0.3293  data_time: 0.0037  lr: 0.0004  max_mem: 2771M
[04/04 11:03:28 d2.utils.events]:  eta: 0:09:02  iter: 8359  total_loss: 0.1491  loss_cls: 0.0134  loss_box_reg: 0.03823  loss_mask: 0.09084  loss_rpn_cls: 9.668e-05  loss_rpn_loc: 0.004961  time: 0.3292  data_time: 0.0036  lr: 0.0004  max_mem: 2771M
[04/04 11:03:35 d2.utils.events]:  eta: 0:08:55  iter: 8379  total_loss: 0.1695  loss_cls: 0.01746  loss_box_reg: 0.03625  loss_mask: 0.09296  loss_rpn_cls: 0.000134  loss_rpn_loc: 0.004694  time: 0.3292  data_time: 0.0035  lr: 0.0004  max_mem: 2771M
[04/04 11:03:41 d2.utils.events]:  eta: 0:08:48  iter: 8399  total_loss: 0.1885  loss_cls: 0.01646  loss_box_reg: 0.06686  loss_mask: 0.08766  loss_rpn_cls: 0.0001205  loss_rpn_loc: 0.003765  time: 0.3292  data_time: 0.0035  lr: 0.0004  max_mem: 27

[04/04 11:06:55 d2.utils.events]:  eta: 0:05:37  iter: 8979  total_loss: 0.2459  loss_cls: 0.03388  loss_box_reg: 0.09129  loss_mask: 0.09622  loss_rpn_cls: 0.0001558  loss_rpn_loc: 0.005877  time: 0.3294  data_time: 0.0034  lr: 0.0004  max_mem: 2771M
[04/04 11:07:01 d2.utils.events]:  eta: 0:05:30  iter: 8999  total_loss: 0.1635  loss_cls: 0.01697  loss_box_reg: 0.06575  loss_mask: 0.08172  loss_rpn_cls: 0.0001054  loss_rpn_loc: 0.004351  time: 0.3294  data_time: 0.0037  lr: 0.0004  max_mem: 2771M
[04/04 11:07:08 d2.utils.events]:  eta: 0:05:24  iter: 9019  total_loss: 0.22  loss_cls: 0.02547  loss_box_reg: 0.07594  loss_mask: 0.1178  loss_rpn_cls: 0.0001407  loss_rpn_loc: 0.005228  time: 0.3295  data_time: 0.0034  lr: 0.0004  max_mem: 2771M
[04/04 11:07:14 d2.utils.events]:  eta: 0:05:17  iter: 9039  total_loss: 0.1227  loss_cls: 0.01561  loss_box_reg: 0.03727  loss_mask: 0.07807  loss_rpn_cls: 0.0001192  loss_rpn_loc: 0.003197  time: 0.3294  data_time: 0.0033  lr: 0.0004  max_mem: 2

[04/04 11:10:29 d2.utils.events]:  eta: 0:02:06  iter: 9619  total_loss: 0.2022  loss_cls: 0.02265  loss_box_reg: 0.07237  loss_mask: 0.09287  loss_rpn_cls: 0.0001498  loss_rpn_loc: 0.00488  time: 0.3297  data_time: 0.0036  lr: 0.0004  max_mem: 2771M
[04/04 11:10:35 d2.utils.events]:  eta: 0:01:59  iter: 9639  total_loss: 0.1835  loss_cls: 0.01761  loss_box_reg: 0.06006  loss_mask: 0.1063  loss_rpn_cls: 8.674e-05  loss_rpn_loc: 0.004996  time: 0.3297  data_time: 0.0037  lr: 0.0004  max_mem: 2771M
[04/04 11:10:42 d2.utils.events]:  eta: 0:01:53  iter: 9659  total_loss: 0.1593  loss_cls: 0.01538  loss_box_reg: 0.04853  loss_mask: 0.0833  loss_rpn_cls: 0.0001116  loss_rpn_loc: 0.00359  time: 0.3297  data_time: 0.0035  lr: 0.0004  max_mem: 2771M
[04/04 11:10:48 d2.utils.events]:  eta: 0:01:46  iter: 9679  total_loss: 0.1701  loss_cls: 0.01989  loss_box_reg: 0.0535  loss_mask: 0.09114  loss_rpn_cls: 0.0001352  loss_rpn_loc: 0.006734  time: 0.3297  data_time: 0.0038  lr: 0.0004  max_mem: 277

In [7]:
# set the testing threshold for this model
#cfg.OUTPUT_DIR = "../Dataset/outputv2"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

In [8]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
predictor = DefaultPredictor(cfg)

[04/04 11:12:37 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia